In [1]:
import pandas as pd
from sklearn.cluster import KMeans
from patternly.detection import AnomalyDetection

data = pd.read_csv("./data1", low_memory=False)
seqs = pd.concat(
    [data[(1000 * i):(1000 * (i + 1))].reset_index(drop=True) for i in range(int(data.shape[0]/1000))], 
    axis=1
)
seqs = seqs.T
seqs.index = range(seqs.shape[0])
seqs

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,1,0,0,1,0,1,0,1,1,...,1,0,1,0,1,0,1,1,0,1
1,0,1,0,1,0,1,0,1,0,0,...,1,0,1,0,1,0,0,1,0,1
2,0,1,0,1,0,1,1,0,1,0,...,0,1,0,1,0,1,0,1,0,1
3,1,0,1,0,1,0,1,1,0,1,...,0,1,0,1,0,1,1,0,1,1
4,0,1,0,0,1,0,1,1,0,1,...,0,1,0,1,0,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,0,1,0,1,0,1,0,0,1,0,...,1,0,0,1,0,1,0,1,0,1
236,0,1,0,0,1,0,1,0,0,1,...,0,1,0,1,0,1,0,1,1,0
237,1,0,1,0,1,0,1,1,0,1,...,0,1,1,0,0,1,0,1,1,0
238,0,1,0,0,0,1,0,1,0,0,...,0,1,0,1,0,1,0,0,1,0


In [2]:
kmeans = KMeans(n_clusters=2)
pipeline = AnomalyDetection(anomaly_sensitivity=2, clustering_alg=kmeans, quantize=False, eps=0.1, verbose=True)
pipeline.fit(seqs)

predictions = pd.DataFrame(pipeline.predict())
anomalies = predictions[predictions[0] == True]
anomalies
print(list(anomalies.index))

Calculating distance matrix...
Clustering distance matrix...
Writing cluster 1/2 to file...
Writing cluster 2/2 to file...
Generating cluster PFSA 1/2...
Generating cluster PFSA 2/2...
Calculating cluster PFSA means and stds...
[27, 96, 200, 209, 239]


In [3]:
pipeline.predict(seqs.iloc[239], clean=False)

True

In [4]:
pipeline.save_model()

In [5]:
new_pipeline = AnomalyDetection().load_model()

predictions = pd.DataFrame(new_pipeline.predict(seqs))
anomalies = predictions[predictions[0] == True]
anomalies
print(list(anomalies.index))

[27, 96, 200, 209, 239]


In [6]:
from IPython.display import Image, display
from IPython.core.display import HTML
for i, file in enumerate(pipeline.dot_files):
    print(f"Cluster {i} PFSA")
    display(Image(url=f"{file}.png", width=300))

Cluster 0 PFSA


Cluster 1 PFSA
